In [1]:
import pandas as pd
import matplotlib
import numpy as np

# loading the feathers generated by the parser script

In [2]:
PATH_ARTICLE = './feathers/article.feather'
PATH_PROCEEDING = './feathers/inproceedings.feather'
PATH_INPROCEEDING = './feathers/proceedings.feather'
df_article = pd.read_feather('./feathers/article.feather')
df_inproceedings = pd.read_feather('./feathers/inproceedings.feather')
df_proceedings = pd.read_feather('./feathers/proceedings.feather')


# splitting the string generated by the parser script which contains all the values of of authors, editors based on " ---" and then exploding it

In [3]:
df_article = df_article.iloc[3:]
df_article['author'] = df_article[df_article['author'].isnull()==False]['author'].str.split(' ---')
article_Key_Name = df_article[['key','author']].explode('author').reset_index(drop=True)
number_article = article_Key_Name.shape[0]

In [4]:
df_proceedings['editor'] = df_proceedings[df_proceedings['editor'].isnull()==False]['editor'].str.split(' ---')
editor_Key_Name = df_proceedings[['key','editor']].explode('editor').reset_index(drop=True)
number_proceedings = editor_Key_Name.shape[0]


In [5]:
df_inproceedings['author'] = df_inproceedings[df_inproceedings['author'].isnull()==False]['author'].str.split(' ---')
inproceedings_Key_Name = df_inproceedings[['key','author']].explode('author').reset_index(drop=True)
number_inproceedings = inproceedings_Key_Name.shape[0]

# Generating the ID for each author/editor by merging all 3 tables, putting same id for same academic

In [53]:
#defining the ranges of the authors/editors
start_articles = 0
end_articles = number_article
start_proceedings = end_articles
end_proceedings = start_proceedings+number_proceedings
start_inproceedings = end_proceedings
end_inproceedings = start_inproceedings+number_inproceedings

In [54]:
# rename the columns of editor_Key_Name from editor to author
editor_Key_Name.columns = ['key','author']
merged = pd.concat([article_Key_Name,editor_Key_Name,inproceedings_Key_Name])

In [55]:
def generate_id(df,column_name):
    df[column_name] = df[column_name].astype('category')
    df[column_name+"_id"]=df[column_name].cat.codes
    return df

In [56]:
#generate the academic id for each author/editor
merged['author'] = merged['author'].astype('category')
merged['author_id']=merged["author"].cat.codes
merged

,key,author,author_id
0,conf/www/HeMLYW07,Hai He,972464
1,conf/www/HeMLYW07,Weiyi Meng,2885951
2,conf/www/HeMLYW07,Yiyao Lu,3022187
3,conf/www/HeMLYW07,Clement T. Yu,517554
4,conf/www/HeMLYW07,Zonghuan Wu,3140055
...,...,...,...
10253077,series/sapere/Besold13,Tarek Richard Besold,2694502
10253078,series/sapere/Steiner13,Pierre Steiner,2183680
10253079,series/sapere/Armstrong13,Stuart Armstrong,2622144
10253080,series/sapere/Freed13,Sam Freed,2417448


## generate academic table

In [57]:
academic = merged[['author_id','author']].drop_duplicates().reset_index(drop=True)
academic.columns = ['academic_id','name']
academic.to_feather('./feathers/academic.feather')

In [58]:
Author_inproceedings = merged[start_inproceedings:end_inproceedings][['author_id','key']].drop_duplicates().reset_index(drop=True)
Author_inproceedings.columns = ['academic_id','improceedings_id']
Author_inproceedings.to_feather('./feathers/author_inproceedings.feather')
Author_inproceedings

,academic_id,improceedings_id
0,249289,www/org/mitre/future
1,2215781,conf/coopis/ChenD00
2,2797674,conf/coopis/ChenD00
3,2683243,conf/coopis/AbdellatifCL04
4,737328,conf/coopis/AbdellatifCL04
...,...,...
10253012,2694502,series/sapere/Besold13
10253013,2183680,series/sapere/Steiner13
10253014,2622144,series/sapere/Armstrong13
10253015,2417448,series/sapere/Freed13


### Generated Editor_proceedings Table

In [59]:
Editor_proceedings = merged[start_proceedings:end_proceedings][['author_id','key']].drop_duplicates().reset_index(drop=True)
Editor_proceedings.columns = ['academic_id','proceedings_id']
Editor_proceedings.to_feather('./feathers/editor_proceedings.feather')
Editor_proceedings

,academic_id,proceedings_id
0,144644,reference/genetic/2015
1,144606,reference/genetic/2015
2,524422,reference/genetic/2015
3,352928,reference/med/2013
4,204395,reference/med/2013
...,...,...
138050,1868829,series/ihis/2015bpm2
138051,2839266,series/sapere/2013-5
138052,644195,tr/trier/MI99-17
138053,2053838,tr/trier/MI99-17


### Generate Editor_proceedings table

In [ ]:
Author_Article = merged[start_articles:end_articles][['author_id','key']].drop_duplicates().reset_index(drop=True)
Author_Article.columns = ['academic_id','article_id']
Author_Article.to_feather('./feathers/author_article.feather')

### Generating rest of the tables

In [ ]:
# remove first 3 rows , since they are not reall data
articles = df_article.drop(['author'],axis=1).iloc[3:].reset_index(drop=True)[['key','title','journal','year','volume','number']]
articles

In [62]:

# generate their journals
# Cleaning the data for CoRR : group all the CoRR article under one category
articles['journal']=articles['journal'].map(lambda x: x if x is not None else 'CoRR')
articles['journal'] = articles['journal'].astype('category')
articles["journal_id"]= articles['journal'].cat.codes
articles

,key,title,journal,journal_id
0,conf/www/RamamohanaraoF07,Patterns Based Classifiers.,World Wide Web,1936
1,conf/www/ZhaiL07,Extracting Web Data Using Instance-Based Learn...,World Wide Web,1936
2,conf/www/NguKN07,Editorial.,World Wide Web,1936
3,conf/www/Ericsson07,The Effects of XML Compression on SOAP Perform...,World Wide Web,1936
4,conf/www/MartinBMMPSMSS07,Bringing Semantics to Web Services with OWL-S.,World Wide Web,1936
...,...,...,...,...
2987941,tr/gte/TR-0244-12-93-165,MetaObject Protocol Concepts for a RISC Object...,GTE Laboratories Incorporated,545
2987942,tr/gte/TR-0146-06-91-165,Towards a Transaction Management System for DOM.,GTE Laboratories Incorporated,545
2987943,tr/gte/TR-0222-10-92-165,DARWIN: On the Incremental Migration of Legacy...,GTE Laboratories Incorporated,545
2987944,tr/dec/SRC1997-018,"The 1995 SQL Reunion: People, Project, and Pol...",Digital System Research Center Report,395


In [63]:
journals_table = articles[['journal_id','journal']].drop_duplicates().reset_index(drop=True)
journals_table.to_feather('journals_table.feather')

In [64]:
articles_table = articles[['key','title','journal_id']]
articles_table.to_feather('./feathers/articles_table.feather')


In [65]:
df_proceedings['publisher'] = df_proceedings[df_proceedings['publisher'].isnull()==False]['publisher'].str.split('/')
get_publishers_df = df_proceedings.explode('publisher').reset_index(drop=True).drop(['editor'],axis=1)
publisher_table = generate_id(get_publishers_df,'publisher')[['publisher_id','publisher']].reset_index(drop=True)
publisher_table.to_feather('./feathers/publisher_table.feather')

In [66]:
publisher_proceeding=get_publishers_df[['publisher_id','key']].drop_duplicates().reset_index(drop=True)
publisher_proceeding.to_feather('./feathers/publisher_proceeding.feather')

In [67]:


x_inproceedings = df_inproceedings.copy()
x_proceedings = df_proceedings.copy()
extract_booktitle = pd.concat([x_inproceedings,x_proceedings]).reset_index(drop=True)

extract_booktitle['booktitle'] = extract_booktitle[extract_booktitle['booktitle'].isnull()==False]['booktitle'].str.split('@')
extract_booktitle['booktitle'] = extract_booktitle['booktitle'].map(lambda x: x[1:] if type(x) is list  and len(x)>1 else x)
extract_booktitle = extract_booktitle.explode('booktitle').reset_index(drop=True)
extract_booktitle['booktitle'] = extract_booktitle[extract_booktitle['booktitle'].isnull()==False]['booktitle'].str.split('/')
extract_booktitle = extract_booktitle.explode('booktitle').reset_index(drop=True)
generate_id(extract_booktitle,'booktitle')

,key,author,title,pages,year,booktitle,editor,publisher,volume,booktitle_id
0,www/org/mitre/future,[Arnon Rosenthal],The Future of Classic Data Administration: Obj...,None,1998,SWEE,NaN,NaN,NaN,9226
1,conf/coopis/ChenD00,"[Qiming Chen, Umeshwar Dayal]",Multi-Agent Cooperative Transactions for E-Com...,311-322,2000,CoopIS,NaN,NaN,NaN,2378
2,conf/coopis/AbdellatifCL04,"[Takoua Abdellatif, Emmanuel Cecchet, Renaud L...",Evaluation of a Group Communication Middleware...,1571-1589,2004,CoopIS,NaN,NaN,NaN,2378
3,conf/coopis/AbdellatifCL04,"[Takoua Abdellatif, Emmanuel Cecchet, Renaud L...",Evaluation of a Group Communication Middleware...,1571-1589,2004,DOA,NaN,NaN,NaN,2618
4,conf/coopis/AbdellatifCL04,"[Takoua Abdellatif, Emmanuel Cecchet, Renaud L...",Evaluation of a Group Communication Middleware...,1571-1589,2004,ODBASE (2),NaN,NaN,NaN,7689
...,...,...,...,...,...,...,...,...,...,...
3261267,series/ihis/2009hoo,NaN,Handbook on Ontologies,NaN,2009,NaN,"[Steffen Staab, Rudi Studer]",NaN,None,-1
3261268,series/ihis/2015bpm1,NaN,"Handbook on Business Process Management 1, Int...",NaN,2015,NaN,"[Jan vom Brocke, Michael Rosemann]",NaN,None,-1
3261269,series/ihis/2015bpm2,NaN,"Handbook on Business Process Management 2, Str...",NaN,2015,NaN,"[Jan vom Brocke, Michael Rosemann]",NaN,None,-1
3261270,series/sapere/2013-5,NaN,Philosophy and Theory of Artificial Intelligen...,NaN,2013,PT-AI,[Vincent C. Müller],NaN,5,8057


In [68]:
first_id_of_x_proceedings = x_proceedings['key'].iloc[0]
#index of the first row of x_proceedings in extract_booktitle
index_in_new_table = extract_booktitle[extract_booktitle['key']==first_id_of_x_proceedings].index[0]

In [69]:
improceeding_table = extract_booktitle[0:index_in_new_table][['key','title','pages','year']].reset_index(drop=True)
improceeding_booktitle = extract_booktitle[0:index_in_new_table][['key','booktitle_id']].reset_index(drop=True)
proceeding_table = extract_booktitle[index_in_new_table+1:][['key','title','year','volume']].reset_index(drop=True)
proceeding_booktitle = extract_booktitle[index_in_new_table+1:][['key','booktitle_id']].reset_index(drop=True)
booktitles_table = extract_booktitle[['booktitle_id','booktitle']].drop_duplicates().reset_index(drop=True)

#connvert to tables (feathers format)
improceeding_table.to_feather('./feathers/improceeding_table.feather')
improceeding_booktitle.to_feather('./feathers/improceeding_booktitle.feather')
proceeding_table.to_feather('./feathers/proceeding_table.feather')
proceeding_booktitle.to_feather('./feathers/proceeding_booktitle.feather')
booktitles_table.to_feather('./feathers/booktitles_table.feather')
